In [1]:
# default_exp data.load_pd

In [2]:
# hide
import sys

sys.path.append("..")
import pandas as pd

/home/tako/dev/env37/lib/python3.7/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [3]:
# export
from fastseq.core import *
from fastseq.data.external import *
from fastcore.utils import *
from fastcore.imports import *
from fastai2.basics import *
from fastai2.data.transforms import *
from fastai2.tabular.core import *

In [4]:
# hide
from nbdev.showdoc import *

# Data Load Dataframe


> Using the fastai2 `Datasets` to make an time series dataset.

A multivaraite time-series dataloader. 

TODO reduce mem: https://forums.fast.ai/t/how-to-handle-dataframes-too-large-to-fit-in-memory/39208/19

In [5]:
# export
import numpy as np
import pandas as pd

In [6]:
# export
class TSMulti(MultiTuple):pass

## Transform catagorical data

In [7]:
# export
class TensorCatI(TensorBase):pass
class CatSeqI(TensorSeqs):pass
def unpack_list(o, r=None):
    r = ifnone(r,L())
    for a in o:
        if isinstance(a,list) or isinstance(a,L):
            r = unpack_list(a, r)
        else:
            r.append(a)
    return r
            
class CatTfm(Transform):
    def __init__(self, df, cat_cols:[]): # maybe change to proccs
        self.vocab,self.o2i = {},{}
        for i, col in enumerate(L(cat_cols)):
            r = unpack_list(list(df[col]))
            self.vocab[i], self.o2i[i] = uniqueify(r, sort=True, bidir=True)
            
    def encodes(self, o:TensorCat):
        r = []
        for i in self.o2i:
            r.append(self.o2i[i][o.o[i]])            
        return TensorCatI(r, label = o._meta['label'])
    
    def decodes(self, o:TensorCatI): 
        r = []
        for i_cat in self.vocab:
            r.append(self.vocab[i_cat][o[i_cat]])
        return TensorCat(r, label = o._meta.get('label',None))
    
    def encodes(self, o:CatSeq):
        r = []
        for i in self.o2i:
            r.append([])
            for a in o.o[i]:
                r[i].append(self.o2i[i][a])            
        return CatSeqI(r, label = o._meta['label'])
    
    def decodes(self, o:CatSeqI):
        r = []
        for i in self.o2i:
            r.append([])
            for a in o[i]:
                r[i].append(self.vocab[i][a])  
        return CatSeq(r, label = o._meta.get('label',None))
    
    

In [8]:
df = pd.DataFrame({'a':[i for i in 'aabb']})
tmf = CatTfm(df, ['a'])
o_tmf = tmf(TensorCat(['a']))
test_eq_type(o_tmf,TensorCatI([0]))
test_eq(tmf.decode(o_tmf),TensorCat(['a']))

In [9]:
df = pd.DataFrame({'a':[i for i in 'aabb'], 'a_2':[i for i in 'bccb'],})
tmf = CatTfm(df, ['a','a_2'])
o_tmf = tmf(TensorCat(['a','b']))
test_eq_type(o_tmf,TensorCatI([0,0]))
test_eq(tmf.decode(o_tmf),TensorCat(['a','b']))

In [10]:
df = pd.DataFrame({'a':[[i,'a'] for i in 'aabb']})
tmf = CatTfm(df, ['a'])
o = CatSeq([['a','b','a','a']], label='a')
o_tmf = tmf(o)
print(o_tmf)
test_eq(o_tmf, TensorCatI([[0, 1, 0, 0]]))
test_eq(o_tmf._meta['label'], 'a')
dec_o = tmf.decode(o_tmf)
test_eq(dec_o,o)
dec_o

CatSeqI([[0, 1, 0, 0]])


CatSeq([['a', 'b', 'a', 'a']], label = ['a'])

In [11]:
df = pd.DataFrame({'a':[i for i in 'aabb'], 'a_2':[i for i in 'bccb'],})
tmf = CatTfm(df, ['a','a_2'])
o = CatSeq([['a','b','a','a'],['b','b','c','c']], label=['a','a_2'])
o_tmf = tmf(o)
print(o_tmf)
test_eq(o_tmf, CatSeqI([[0, 1, 0, 0],
                        [0, 0, 1, 1]]))
test_eq(o_tmf._meta['label'], ['a','a_2'])
dec_o = tmf.decode(o_tmf)
test_eq(dec_o,o)
dec_o

CatSeqI([[0, 1, 0, 0],
        [0, 0, 1, 1]])


CatSeq([['a', 'b', 'a', 'a'], ['b', 'b', 'c', 'c']], label = ['a', 'a_2'])

In [12]:
# export
class TSMulti_(Tuple):pass

class CatMultiTfm(ItemTransform):
    @delegates(CatTfm.__init__)
    def __init__(self, *args, **kwargs): # maybe change to proccs
        self.f = CatTfm(*args, **kwargs)
        
    def encodes(self, o:TSMulti):
        return TSMulti_(self.f(a) for a in o)
    
    def decodes(self, o:TSMulti_): 
        return TSMulti(self.f.decode(a) for a in o)
    

In [13]:
df = pd.DataFrame({'a':[i for i in 'aabb']})
tmf = CatMultiTfm(df,'a')
print(tmf)
o = TSMulti(TensorBase([1]), TensorCat('a'))
o_tmf = tmf(o)
test_eq(o_tmf,(TensorBase([1]), TensorCatI([0])) )
o_b =tmf.decode(o_tmf)
test_eq(o_b, o)
test_eq(type(o_b), type(TSMulti(1)))

CatMultiTfm: True (TSMulti,object) -> encodes (TSMulti_,object) -> decodes


# Dataloader

## pred

In [14]:
def get_df(length = [100,120]):
    dct = {'x':[],'con_ts_1':[],'con_ts_0':[],'cat_ts_1':[],'cat_ts_0':[],'con_0':[],'con_1':[], 'cat_0':[],'cat_1':[]}
    for i, l in enumerate(length):
        assert int(l/2) == l/2
        dct['x'].append(np.arange(l))
        dct['con_ts_0'].append(np.ones(l)[None,:])
        dct['con_ts_1'].append(pd.Series(np.arange(l)+np.random.randn(l)))
        dct['con_0'].append(np.random.randn())
        dct['con_1'].append(10+np.random.randn()*2)
        dct['cat_ts_0'].append(L(['a','b']*int(l/2)))
        dct['cat_ts_1'].append(L(['y','z']*int(l/2)))
        dct['cat_0'].append(['a','b'][i%2])
        dct['cat_1'].append(['d','c'][i%2])
    return pd.DataFrame(data=dct)
df = get_df([10,12,6])
df

,x,con_ts_1,con_ts_0,cat_ts_1,cat_ts_0,con_0,con_1,cat_0,cat_1
0,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]",0 -0.397057 1 0.742402 2 3.338075 3 3.981841 4 4.482663 5 4.539409 6 5.740296 7 5.815583 8 6.786242 9 10.698944 dtype: float64,"[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]]","[y, z, y, z, y, z, y, z, y, z]","[a, b, a, b, a, b, a, b, a, b]",-0.479305,10.429260,a,d
1,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]",0 -2.100359 1 0.509973 2 3.262948 3 3.883042 4 4.023184 5 3.295666 6 5.306996 7 7.766818 8 6.803480 9 10.602652 10 10.386768 11 12.795048 dtype: float64,"[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]]","[y, z, y, z, y, z, y, z, y, z, y, z]","[a, b, a, b, a, b, a, b, a, b, a, b]",-0.631020,11.233439,b,c
2,"[0, 1, 2, 3, 4, 5]",0 0.127216 1 0.261255 2 2.811576 3 1.925198 4 3.844308 5 3.922653 dtype: float64,"[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0]]","[y, z, y, z, y, z]","[a, b, a, b, a, b]",1.556760,10.280112,a,d


In [15]:
# export
def array2series(o):
    return pd.Series(o.flatten())
def arrays2series(s:pd.Series):
    return pd.Series([array2series(o) for o in s])

In [16]:
df = pd.DataFrame(data={'x':  [np.arange(10.),
                               np.arange(12.)],
                       'ts_0':[np.ones(10)[None,:],
                               np.ones(12)[None,:]],
                       })
df['x'] = arrays2series(df['x'])
df['ts_0'] = arrays2series(df['ts_0'])
test_eq(type(df['ts_0']), pd.Series)
test_eq(type(df['x']), pd.Series)

In [17]:
# hide
df = pd.DataFrame(data={'x':  [np.arange(10.)]*100,
                       'ts_0':[np.ones(10)[None,:]]*100,
                       })
df['x'] = arrays2series(df['x'])
df['ts_0'] = arrays2series(df['ts_0'])
test_eq(type(df['ts_0']), pd.Series)
test_eq(type(df['x']), pd.Series)

In [18]:
# export
def reconize_cols(dataset):
    con_names, cat_names, con_ts_names, cat_ts_names, classes = L(), L(), L(), L(), {}
    for col in dataset.columns:
        t = type(dataset[col].iloc[0])
        if t is pd.core.series.Series:
            con_ts_names.append(col)
        elif t is np.ndarray:
            o = arrays2series(dataset[col])
            dataset[col] = o
            con_ts_names.append(col)
        elif isinstance(dataset[col].iloc[0], str):
            cat_names.append(col)
            classes[col] = uniqueify(list(dataset[col].values))
        elif isinstance(dataset[col].iloc[0], float) or isinstance(dataset[col].iloc[0], int) or t is np.int64:
            con_names.append(col)            
        elif isinstance(dataset[col].iloc[0], L) or isinstance(dataset[col].iloc[0],list):
            cat_ts_names.append(col)
            classes[col] = uniqueify(unpack_list(list(dataset[col])))
        else:
            raise Exception(t)
    return con_names, cat_names, con_ts_names, cat_ts_names, classes, dataset

In [19]:
df = get_df()
con_names, cat_names, con_ts_names, cat_ts_names, classes, dataset = reconize_cols(df)
test_eq(con_names, ['con_0','con_1'])
test_eq(cat_names, ['cat_0','cat_1'])
test_eq(set(con_ts_names), set(['x','con_ts_0','con_ts_1']))
test_eq(set(cat_ts_names), set(['cat_ts_0','cat_ts_1']))
test_eq(classes, {'cat_ts_0': ['a','b'],
                  'cat_ts_1': ['y','z'],
                  'cat_0': ['a','b'],
                  'cat_1': ['d','c']
                 })
con_names, cat_names, con_ts_names, cat_ts_names

((#2) ['con_0','con_1'],
 (#2) ['cat_0','cat_1'],
 (#3) ['x','con_ts_1','con_ts_0'],
 (#2) ['cat_ts_1','cat_ts_0'])

In [20]:
list([L([a for a in o]) for o in df[['cat_ts_0','cat_ts_1']].values])

[(#2) [(#100) ['a','b','a','b','a','b','a','b','a','b'...],(#100) ['y','z','y','z','y','z','y','z','y','z'...]],
 (#2) [(#120) ['a','b','a','b','a','b','a','b','a','b'...],(#120) ['y','z','y','z','y','z','y','z','y','z'...]]]

In [21]:
# export
class PrepDF(TabularProc):
    def setup(self, dl, train_setup):
        # speed up retrival
        dl.con = dl.dataset.loc[:,dl.con_names].values.astype(float)
        dl.cat = [list(dl.dataset.loc[i,dl.cat_names]) for i in range(dl.dataset.shape[0])]
        if len(dl.con_ts_names):
            dl.tsx_con = [np.concatenate([o[None,:] for o in dl.dataset.loc[i,dl.con_ts_names].values]) for i in range(dl.dataset.shape[0])]
        else:
            dl.tsx_con = [np.empty([0,0]) for i in range(dl.dataset.shape[0])]
            
        if len(dl.cat_ts_names):
            dl.tsx_cat = list([L([a for a in o]) for o in df[dl.cat_ts_names].values])
        else:
            dl.tsx_cat = ['']*dl.dataset.shape[0]   
        assert len(dl.cat) == len(dl.tsx_con) == dl.con.shape[0] == len(dl.tsx_cat), f"{len(dl.cat)} == {len(dl.tsx_con)} == {dl.con.shape[0]} == {len(dl.tsx_cat)}"
        return dl

In [22]:
class test_dl(TfmdDL):    
    def __init__(self, dataset:pd.DataFrame, y_name, lookback = 10, horizon=3,step=1,min_seq_len=None):
        con_names, cat_names, con_ts_names, cat_ts_names, classes, dataset = reconize_cols(dataset)  
        store_attr(self,'lookback,horizon,step,y_name,con_names,cat_names,con_ts_names,cat_ts_names,classes,dataset')     
        assert y_name in self.con_ts_names, {k:getattr(self,k) for k in 'con_names,cat_names,con_ts_names,cat_ts_names'.split(',')}
        self.con_ts_names.remove(y_name) 
        PrepDF().setup(self, True)    
        self.min_seq_len = ifnone(min_seq_len, lookback)
        
df = get_df()
dl = test_dl(df, 'x')
test_eq(dl.cat,[['a', 'd'], ['b', 'c']])
test_eq(dl.con.shape,(2,2))
test_eq(dl.tsx_con[0].shape, [2,100])
test_eq(len(dl.tsx_cat), 2)
test_eq(len(dl.tsx_cat[0]), 2)
test_eq(len(dl.tsx_cat[0][0]), 100)
dl.tsx_cat

[(#2) [(#100) ['y','z','y','z','y','z','y','z','y','z'...],(#100) ['a','b','a','b','a','b','a','b','a','b'...]],
 (#2) [(#120) ['y','z','y','z','y','z','y','z','y','z'...],(#120) ['a','b','a','b','a','b','a','b','a','b'...]]]

In [23]:
for cols in [['x'],['x','cat_ts_0'],['x','con_ts_1'],['x','cat_1'],['x','con_0']]:
    print(cols)
    df = get_df()[cols]
    dl = test_dl(df, 'x')


['x']
['x', 'cat_ts_0']
['x', 'con_ts_1']
['x', 'cat_1']
['x', 'con_0']


In [24]:
# export
def same_size_ts(ts:pd.Series, ts_names, _raise = True):
    shapes = {k:ts[k].shape if hasattr(ts[k],'shape') else (len(ts[k]),) for k in ts_names}
    all_same = [[(shapes[c] == shapes[a]) for c in ts_names] for a in ts_names]
    mask = np.array(all_same)
    if _raise:
        assert np.sum(mask) == len(ts_names)**2, shapes
    return np.sum(mask) == len(ts_names)**2
    

In [25]:
df = get_df()
con_names, cat_names, con_ts_names, cat_ts_names, classes, dataset = reconize_cols(df)
same_size_ts(dataset.iloc[0,:], ['x','con_ts_0','con_ts_1','cat_ts_0'])

True

In [26]:
# hide
df = pd.DataFrame(data={'ts_0':[pd.Series(np.arange(10)),
                           pd.Series(np.arange(10))],
                   'ts_1':[pd.Series(np.arange(1,11)),
                           pd.Series(np.arange(1,12))],
                   'var_0':[0.,1.],
                   'con_0':[0,1]})
for i, row in df.iterrows():
    if i == 0:
        test_eq(same_size_ts(row,['ts_0','ts_1'], False),True)
    if i == 1:
        test_eq(same_size_ts(row,['ts_0','ts_1'], False),False)

In [27]:
# export
def make_ids(dl):
    """Make ids if the sequence is shorter than `min_seq_len`, it will drop that sequence."""
    # Slice each time series into examples, assigning IDs to each
    last_id = 0
    n_dropped = 0
    n_needs_padding = 0
    dl._ids = {}
    for i, ts in dl.dataset.iterrows():
        same_size_ts(ts, dl.con_ts_names + dl.cat_ts_names)
        num_examples = (ts[dl.y_name].shape[-1] - dl.lookback - dl.horizon + dl.step) // dl.step
        # Time series shorter than the forecast horizon need to be dropped.
        if ts[dl.y_name].shape[-1] < dl.min_seq_len:
            n_dropped += 1
            continue
        # For short time series zero pad the input
        if ts[dl.y_name].shape[-1] < dl.lookback + dl.horizon:
            n_needs_padding += 1
            num_examples = 1
        for j in range(num_examples):
            dl._ids[last_id + j] = (i, j * dl.step)
        last_id += num_examples

    # Inform user about time series that were too short
    if n_dropped > 0:
        print("Dropped {}/{} time series due to length.".format(
                n_dropped, len(dl.dataset)))

    # Inform user about time series that were short
    if n_needs_padding > 0:
        print("Need to pad {}/{} time series due to length.".format(
                n_needs_padding, len(dl.dataset)))
    # Store the number of training examples
    dl.n = int(dl._ids.__len__() )
    return dl, int(dl._ids.__len__() )
    

In [28]:
df = get_df([14,14,16])
dl = test_dl(df, 'x', lookback= 10,horizon = 2)
dl, n = make_ids(dl)
test_eq(dl._ids,{0: (0, 0),
                 1: (0, 1),
                 2: (0, 2),
                 3: (1, 0),
                 4: (1, 1),
                 5: (1, 2),
                 6: (2, 0),
                 7: (2, 1),
                 8: (2, 2),
                 9: (2, 3),
                 10: (2, 4)
                })

In [32]:
# export
@typedispatch
def get_part_of_ts(x, lookback_id, length, pad=np.mean, t = tensor, **kwargs):
#     if len(x.shape) == 1:
#         x = x[None,:]
#     if isinstance(x[0,0],int):
#         x = x.astype(float)
    if x.shape[-1] < length:
        # If the time series is too short, we pad
        padding = pad(x, -1)
        x = t(np.pad(
            x, # report issue https://github.com/numpy/numpy/issues/15606
            pad_width=((0, 0), (length - x.shape[-1], 0)),
            mode='constant',
            constant_values=padding
        ), **kwargs).float()
        assert x.shape == (x.shape[0],length), f"{x.shape}\t,{lookback_id}, 'tsshape':{x.shape}"
    else:
        x = t(x[:,lookback_id:lookback_id + length], **kwargs).float()
    return x


In [33]:
x = np.vstack([np.arange(10.)]*2)
test_eq(get_part_of_ts(x, 2, 5), torch.cat([torch.arange(2,7.)[None,:]]*2))
test_eq(get_part_of_ts(x, 0, 11), tensor([[4.5, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
                                          [4.5, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9]]))

# report issue https://github.com/numpy/numpy/issues/15606
x = np.arange(10.)[None,:]
test_eq(get_part_of_ts(x, 0, 11), tensor([[4.5, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9]]))

In [34]:
# hide
x = np.arange(10.)[None,:]
o =get_part_of_ts(x, 0, 11, t = TensorBase, label='a')
test_eq_type(o, TensorBase([[4.5, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9]])) 
test_eq(o._meta, {'label':'a'})

In [35]:
# export
@typedispatch
def get_part_of_ts(x:L, lookback_id, length, t = L, **kwargs):
    if len(x[0]) < length:
        # If the time series is too short, we pad
        padding = [o[-1] for o in x]
        pad_len = length - len(x[0])
        x = t(L(o[lookback_id:lookback_id + length] + [padding[i]]*pad_len) for i,o in enumerate(x))
    else:
        x = t([o[lookback_id:lookback_id + length] for o in x], **kwargs)
    return x

In [36]:
x = L(['a','b']*5,['z','x']*5)
o =get_part_of_ts(x, 1, 5)
test_eq(len(o), 2)
for a in o:
    test_eq(len(a),5)
test_eq(type(o),L)

In [37]:
x = L(['a','b']*2, ['z','x']*2)
o =get_part_of_ts(x, 0, 5)
test_eq(len(o), 2)
for a in o:
    test_eq(len(a),5)
test_eq(type(o),L)

In [42]:
# export
from fastseq.core import *
def get_id(dl, ts_id, lookback_id):
    y = get_part_of_ts(dl.dataset.loc[ts_id, dl.y_name].values[None,:], lookback_id, dl.lookback + dl.horizon,
                       t = TensorSeqs, label=[dl.y_name + '_y'], m=['g'])
    x = TensorSeqs(y[:,:dl.lookback], label=[dl.y_name + '_x'], m=['g'])
    if len(dl.con_ts_names):
        tsx_con = get_part_of_ts(dl.tsx_con[ts_id], lookback_id, dl.lookback + dl.horizon,
                             t = TensorSeqs, label=dl.con_ts_names)
    else: tsx_con = TensorSeqs(np.empty([0]), label=dl.con_ts_names)
    if len(dl.cat_ts_names):
        tsx_cat = get_part_of_ts(dl.tsx_cat[ts_id], lookback_id, dl.lookback + dl.horizon,
                             t = CatSeq, label=dl.cat_ts_names)
    else: tsx_cat = CatSeq('', label=dl.cat_ts_names)
        
    r = [x, tsx_con, tsx_cat]
    r.append(TensorCat(dl.cat[ts_id], label=dl.cat_names))
    r.append(TensorCon(dl.con[ts_id,:], label=dl.con_names))      
    r.append(y)
    return tuple(r)

In [44]:
df = get_df([14,14,16])
dl = test_dl(df, 'x', lookback= 10,horizon = 2)
dl, n = make_ids(dl)
for k, (ts_id, lookback_id) in dl._ids.items():
    r = get_id(dl, ts_id, lookback_id)
    break
r
    

(TensorSeqs([[0., 1., 2., 3., 4., 5., 6., 7., 8., 9.]]),
 TensorSeqs([[ 0.7595, -0.2018,  3.6567,  3.9557,  2.3628,  4.5180,  5.8373,  7.1024,
           7.2677,  7.7250,  9.2480, 10.4108],
         [ 1.0000,  1.0000,  1.0000,  1.0000,  1.0000,  1.0000,  1.0000,  1.0000,
           1.0000,  1.0000,  1.0000,  1.0000]]),
 CatSeq([(#12) ['y','z','y','z','y','z','y','z','y','z'...], (#12) ['a','b','a','b','a','b','a','b','a','b'...]], label = ['cat_ts_1', 'cat_ts_0']),
 TensorCat(['a', 'd'], label = ['cat_0', 'cat_1']),
 TensorCon([1.1779, 9.2421]),
 TensorSeqs([[ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11.]]))

## TfmDL

In [45]:
# export
@delegates()
class DfDataLoader(TfmdDL):
    def __init__(self, dataset:pd.DataFrame, y_name:str, horizon:int, lookback=72, step=1,
                 min_seq_len=None, procs = None, train = True, **kwargs):
        con_names, cat_names, con_ts_names, cat_ts_names, classes, dataset = reconize_cols(dataset)  
        store_attr(self,'horizon,lookback,step,y_name,con_names,cat_names,con_ts_names,cat_ts_names,classes,dataset')     
        assert y_name in self.con_ts_names, {k:getattr(self,k) for k in 'con_names,cat_names,con_ts_names,cat_ts_names'.split(',')}
        self.con_ts_names.remove(y_name)         
        self.min_seq_len = ifnone(min_seq_len, lookback)
        self, n = make_ids(self)
        kwargs['after_item'] = kwargs.get('after_item', CatMultiTfm(dataset, self.cat_names+self.cat_ts_names))
        super().__init__(dataset=self.dataset, **kwargs)
        self.n = n
        self.procs = Pipeline(PrepDF() +L(procs), as_item=True)
        self.procs.setup(self, train)            

    @delegates(TfmdDL.new)
    def new(self, dataset=None, cls=None, **kwargs):
        for k,v in {k:getattr(self,k) for k in ['horizon', 'lookback', 'step']}.items():
            if k not in kwargs:
                kwargs[k] = v
        res = super().new(dataset = dataset,cls= cls, y_name= self.y_name, **kwargs)
        res, n = make_ids(res)
        res.n = n
        return res    
    
    def create_item(self, idx):
        if idx>=self.n:
            raise IndexError
        ts_id, lookback_id = self._ids[idx]
        r  = get_id(self, ts_id, lookback_id) 
        return TSMulti(r)    

The data will be stored in pandas DataFrames. This can be use for time series by nesting series into a cell. An example is shown below: 

In [46]:
# hide
from nbdev.export import *
notebook2script()

Converted 00_core.ipynb.
Converted 01_data.external.ipynb.
Converted 020_data.load_pd.ipynb.
Converted 02_data.load.ipynb.
Converted 030_data.core_pd.ipynb.
Converted 03_data.core.ipynb.
Converted 050_nbeats.models_multi.ipynb.
Converted 05_nbeats.models.ipynb.
Converted 06_nbeats.callbacks.ipynb.
Converted 07_nbeats.learner.ipynb.
Converted 08_nbeats.interpret.ipynb.
Converted 11_metrics.ipynb.
Converted 12_compare.ipynb.
Converted index.ipynb.
Converted tab.model.ipynb.


In [47]:
git_add('020_data.load_pd.ipynb')

Converted 020_data.load_pd.ipynb.


['/home/tako/dev/fastseq/fastseq/data/load_pd.py',
 '/home/tako/dev/fastseq/nbs/020_data.load_pd.ipynb']